In [1]:
import pandas as pd
from lxml import etree
import xml.etree.ElementTree as ET

In [3]:
file_path = r'C:\Users\amank\Downloads\Fithack\export.xml'

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# import xml.etree.ElementTree as ET

# # Load and parse the XML file
# tree = ET.parse(r'C:\Users\amank\Downloads\Fithack\export.xml')
# root = tree.getroot()

In [4]:
# print(root)

In [5]:
# for i, child in enumerate(root):
#     if i >= 15:
#         break
#     print(child.tag, child.attrib)

In [6]:
# for child in root:
#     print(child.tag, child.attrib)

In [7]:
# # Initialize a list to store the data
# data = []

# # Iterate through the XML elements and extract data
# for i, child in enumerate(root):
#     # print(child.tag, child.attrib)
#     row_data = {}
#     row_data[child.tag] = child.attrib
#     data.append(row_data)

In [8]:
# len(data)

In [9]:
# Personal = [item['Me'] for item in data if 'Me' in item]

In [10]:
# personal_df = pd.DataFrame(Personal)
# personal_df = personal_df.rename(columns=lambda x: x.replace('HKCharacteristicTypeIdentifier', ''))
# personal_df

In [11]:
# records = [item['Record'] for item in data if 'Record' in item]

In [12]:
# len(records)

In [13]:
# import pandas as pd
# df = pd.DataFrame(records)

In [14]:
# df.head()

In [15]:
# df.to_csv(r'C:\Users\amank\Downloads\Fithack\export.csv', index=False)

In [16]:
# df.columns

In [17]:
# tree = ET.parse(file_path)
# root = tree.getroot()

# extracted_data = []

# def extract_element_data(element):
#     element_info = {
#         'tag': element.tag,
#         'attributes': element.attrib,
#         'children': []
#     }

#     for child in element:
#         child_data = extract_element_data(child)
#         element_info['children'].append(child_data) 

#     if element.tag == 'Record':
#         extracted_data.append(element_info)

#     return element_info

# extracted_dict = extract_element_data(root)

In [18]:
# extracted_dict

In [19]:
# dataframes = {}

# for child in extracted_dict['children']:
#     tag = child['tag']
#     attributes = child['attributes']
    
#     if tag not in dataframes:
#         dataframes[tag] = []
    
#     dataframes[tag].append(attributes)

# for tag, records in dataframes.items():
#     dataframes[tag] = pd.DataFrame(records)

# for tag, df in dataframes.items():
#     print(f"DataFrame for tag: {tag}")
#     display(df.head())

In [20]:
# dataframes['ActivitySummary']

In [21]:
# from collections import defaultdict

# # Initialize sets to store unique values
# unique_tags = set()
# unique_types = set()
# unique_children = set()
# unique_grandchildren = set()
# unique_times = set()

# # Function to recursively extract data
# def extract_data(element, level=0):
#     tag = element['tag']
#     attributes = element['attributes']
#     children = element['children']

#     unique_tags.add(tag)
#     if 'type' in attributes:
#         unique_types.add(attributes['type'])
#     if 'creationDate' in attributes:
#         unique_times.add(attributes['creationDate'])

#     if level == 0:
#         for child in children:
#             unique_children.add(child['tag'])
#             extract_data(child, level + 1)
#     elif level == 1:
#         for grandchild in children:
#             unique_grandchildren.add(grandchild['tag'])
#             extract_data(grandchild, level + 1)

# # Extract data from the root element
# extract_data(extracted_dict)

# # Print the unique values
# print("Unique Tags:", unique_tags)
# print("Unique Types:", unique_types)
# print("Unique Children:", unique_children)
# print("Unique Grandchildren:", unique_grandchildren)
# print("Unique Times:", unique_times)

In [22]:
class parseXML:
    def __init__(self, file_path):
        self.file_path = file_path
        self.root = self._parse_xml()

    def _parse_xml(self):
        try:
            parser = etree.XMLParser(recover=True)
            tree = etree.parse(self.file_path, parser)
            return tree.getroot()
        except etree.XMLSyntaxError as e:
            print(f"XML Syntax Error: {e}")
            return None

    def _extract_data(self, tag):
        elements = [dict(child.attrib) for child in self.root if child.tag == tag]
        return pd.DataFrame(elements) if elements else pd.DataFrame()

    def _extract_instantaneous_bpm(self, record_element):
        bpm_data = []
        hrv_metadata_list = record_element.find('HeartRateVariabilityMetadataList')
        
        if hrv_metadata_list is not None:
            for bpm_entry in hrv_metadata_list.findall('InstantaneousBeatsPerMinute'):
                bpm_data.append({
                    'time': bpm_entry.attrib.get('time'),
                    'bpm': bpm_entry.attrib.get('bpm')
                })
        return bpm_data

    def _extract_algorithm_version(self, record_element):
        metadata_list = record_element.findall('MetadataEntry')
        for entry in metadata_list:
            if entry.attrib.get('key') == 'HKAlgorithmVersion':
                return entry.attrib.get('value')
        return None

    def records_data(self):
        records = []
        for record in self.root.findall('Record'):
            record_attribs = dict(record.attrib)

            bpm_data = self._extract_instantaneous_bpm(record)
            algorithm_version = self._extract_algorithm_version(record)

            if bpm_data:
                for bpm_entry in bpm_data:
                    combined_record = record_attribs.copy() 
                    combined_record.update(bpm_entry)
                    combined_record['HKAlgorithmVersion'] = algorithm_version
                    records.append(combined_record)
            else:
                record_attribs['HKAlgorithmVersion'] = algorithm_version 
                records.append(record_attribs)

        return pd.DataFrame(records) if records else pd.DataFrame()

    def workouts_data(self):
        return self._extract_data('Workout')

    def activity_summaries_data(self):
        return self._extract_data('ActivitySummary')

parser = parseXML(file_path)

In [23]:
df_records = parser.records_data()
df_records

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,HKAlgorithmVersion,device,time,bpm
0,HKQuantityTypeIdentifierHeight,rahul’s iPhone,16.0.3,cm,2022-10-23 12:29:20 +0530,2022-10-23 12:29:20 +0530,2022-10-23 12:29:20 +0530,178,None,NaN,NaN,NaN
1,HKQuantityTypeIdentifierBodyMass,Health,17.1.2,kg,2023-12-24 20:44:13 +0530,2023-12-24 20:44:12 +0530,2023-12-24 20:44:12 +0530,85,None,NaN,NaN,NaN
2,HKQuantityTypeIdentifierBodyMass,rahul’s iPhone,17.0.2,kg,2023-09-28 13:22:05 +0530,2023-09-28 13:22:05 +0530,2023-09-28 13:22:05 +0530,86,None,NaN,NaN,NaN
3,HKQuantityTypeIdentifierBodyMass,rahul’s iPhone,16.0.3,kg,2022-10-23 12:29:20 +0530,2022-10-23 12:29:20 +0530,2022-10-23 12:29:20 +0530,92,None,NaN,NaN,NaN
4,HKQuantityTypeIdentifierBodyMass,iPhone,17.3.1,kg,2024-03-18 10:04:04 +0530,2024-03-18 10:04:04 +0530,2024-03-18 10:04:04 +0530,82,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1042711,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,2,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",3:10:26.99 PM,117
1042712,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,2,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",3:10:27.49 PM,120
1042713,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,2,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",3:10:27.97 PM,124
1042714,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,2,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",3:10:28.48 PM,118


In [24]:
df_workout = parser.workouts_data()
df_workout

,workoutActivityType,duration,durationUnit,sourceName,sourceVersion,device,creationDate,startDate,endDate
0,HKWorkoutActivityTypeWalking,57.16203097899755,min,rahul’s Apple Watch,10.0.2,"<<HKDevice: 0x30257cc80>, name:Apple Watch, ma...",2023-09-29 21:22:50 +0530,2023-09-29 20:16:41 +0530,2023-09-29 21:22:48 +0530
1,HKWorkoutActivityTypeWalking,23.7147382736206,min,rahul’s Apple Watch,10.0.2,"<<HKDevice: 0x30257cc80>, name:Apple Watch, ma...",2023-09-29 21:49:28 +0530,2023-09-29 21:25:44 +0530,2023-09-29 21:49:27 +0530
2,HKWorkoutActivityTypeWalking,2.773285017410914,min,rahul’s Apple Watch,10.0.2,"<<HKDevice: 0x30257cc80>, name:Apple Watch, ma...",2023-09-29 21:53:50 +0530,2023-09-29 21:51:03 +0530,2023-09-29 21:53:49 +0530
3,HKWorkoutActivityTypeWalking,60.07199061711629,min,rahul’s Apple Watch,10.0.2,"<<HKDevice: 0x30257cc80>, name:Apple Watch, ma...",2023-09-30 22:33:08 +0530,2023-09-30 21:33:01 +0530,2023-09-30 22:33:06 +0530
4,HKWorkoutActivityTypeWalking,39.43156214753787,min,rahul’s Apple Watch,10.0.2,"<<HKDevice: 0x30257cc80>, name:Apple Watch, ma...",2023-10-01 07:28:37 +0530,2023-10-01 06:48:14 +0530,2023-10-01 07:27:40 +0530
...,...,...,...,...,...,...,...,...,...
272,HKWorkoutActivityTypeWalking,15.17823849717776,min,rahul’s Apple Watch,10.6,"<<HKDevice: 0x302591a40>, name:Apple Watch, ma...",2024-09-08 22:14:24 +0530,2024-09-08 21:59:12 +0530,2024-09-08 22:14:23 +0530
273,HKWorkoutActivityTypeWalking,22.95050600369771,min,rahul’s Apple Watch,10.6,"<<HKDevice: 0x302591a40>, name:Apple Watch, ma...",2024-09-09 22:05:01 +0530,2024-09-09 21:42:03 +0530,2024-09-09 22:05:00 +0530
274,HKWorkoutActivityTypeWalking,11.1648644665877,min,rahul’s Apple Watch,10.6,"<<HKDevice: 0x302591a40>, name:Apple Watch, ma...",2024-09-09 22:23:01 +0530,2024-09-09 22:11:50 +0530,2024-09-09 22:23:00 +0530
275,HKWorkoutActivityTypeWalking,45.43507585128148,min,rahul’s Apple Watch,10.6.1,"<<HKDevice: 0x3025eff20>, name:Apple Watch, ma...",2024-09-13 22:01:07 +0530,2024-09-13 21:15:40 +0530,2024-09-13 22:01:06 +0530


In [25]:
df_activities = parser.activity_summaries_data()
df_activities

,dateComponents,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal
0,2022-10-23,238.864,440,kcal,0,0,0,0,0,0
1,2022-10-24,161.641,440,kcal,0,0,0,0,0,0
2,2022-10-25,49.4392,440,kcal,0,0,0,0,0,0
3,2022-10-26,254.831,440,kcal,0,0,0,0,0,0
4,2022-10-27,275.953,440,kcal,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
684,2024-09-16,521.192,550,kcal,0,0,21,60,15,12
685,2024-09-17,463.587,550,kcal,0,0,1,60,15,12
686,2024-09-18,515.612,550,kcal,0,0,7,60,17,12
687,2024-09-19,478.05,550,kcal,0,0,1,60,15,12


In [5]:
# class parseXML:
#     def __init__(self, file_path):
#         self.file_path = file_path
#         self.root = self._parse_xml()

#     def _parse_xml(self):
#         try:
#             parser = etree.XMLParser(recover=True)
#             tree = etree.parse(self.file_path, parser)
#             return tree.getroot()
#         except etree.XMLSyntaxError as e:
#             print(f"XML Syntax Error: {e}")
#             return None

#     def _extract_element_data(self, element):
#         element_data = {f"{element.tag}_text": element.text.strip() if element.text else None}
#         element_data.update(element.attrib)
        
#         for child in element:
#             child_data = self._extract_element_data(child)
#             for key, value in child_data.items():
#                 element_data[f"{element.tag}_{key}"] = value

#         return element_data

#     def parse_to_dataframes(self):
#         data_by_tag = {}

#         for element in self.root.iter():
#             tag = element.tag
#             element_data = self._extract_element_data(element)
            
#             if tag not in data_by_tag:
#                 data_by_tag[tag] = []

#             data_by_tag[tag].append(element_data)

#         dataframes = {
#             tag: self._remove_nan_columns(pd.DataFrame(records))
#             for tag, records in data_by_tag.items()
#         }
#         return dataframes
    
#     def _remove_nan_columns(self, df):
#         return df.dropna(axis=1, how='all')

# parser = parseXML(file_path)

# dataframes = parser.parse_to_dataframes()

In [4]:
# dataframes['Workout'].keys()

In [3]:
# dataframes['Record'].keys()

In [2]:
# dataframes['InstantaneousBeatsPerMinute']

In [1]:
# dataframes['ActivitySummary']

In [3]:
def read_xml(file_path):
    return ET.parse(file_path).getroot()

# file_path = r'/content/export.xml' 

root = read_xml(file_path)

def extract_data(child):
    records_data = []  

    base_data = child.attrib.copy()
    if not child:
        for metadata_entry in child.findall('.//MetadataEntry'):
            base_data[f'MetadataEntry_{metadata_entry.attrib["key"]}'] = metadata_entry.attrib.get('value')
        records_data.append(base_data)
    for metadata_entry in child.findall('.//MetadataEntry'):
        base_data[f'MetadataEntry_{metadata_entry.attrib["key"]}'] = metadata_entry.attrib.get('value')

    for nested_element in child:
        data = base_data.copy()
        data['element_type'] = nested_element.tag
        
        for attr_key, attr_value in nested_element.attrib.items():
            data[f'{nested_element.tag}_{attr_key}'] = attr_value
        
        for sub_element in nested_element:
            sub_element_data = data.copy()
            sub_element_data['sub_element_type'] = sub_element.tag
            for sub_attr_key, sub_attr_value in sub_element.attrib.items():
                sub_element_data[f'{sub_element.tag}_{sub_attr_key}'] = sub_attr_value
            records_data.append(sub_element_data)
        
        if not nested_element:
            records_data.append(data)

    return pd.DataFrame(records_data)



In [ ]:
# import xml.etree.ElementTree as ET
# import pandas as pd

# class XMLParser:
#     def __init__(self, xml_file):
#         self.xml_file = xml_file
#         self.tree = ET.parse(self.xml_file)
#         self.root = self.tree.getroot()
#         self.extracted_data = []

#     def extract_element_data(self, element=None):
#         if element is None:
#             element = self.root
        
#         element_info = {
#             'tag': element.tag,
#             'attributes': element.attrib,
#             'children': []
#         }

#         for child in element:
#             child_data = self.extract_element_data(child)
#             element_info['children'].append(child_data)

#         if element.tag == 'Record':
#             self.extracted_data.append(element_info)

#         return element_info

#     def convert_to_dataframes(self):
#         extracted_dict = self.extract_element_data()

#         dataframes = {}
#         for child in extracted_dict['children']:
#             tag = child['tag']
#             attributes = child['attributes']

#             if tag not in dataframes:
#                 dataframes[tag] = []

#             dataframes[tag].append(attributes)

#         for tag, records in dataframes.items():
#             dataframes[tag] = pd.DataFrame(records)

#         return dataframes

# xml_file = r'C:\Users\amank\Downloads\Fithack\export.xml'
# parser = XMLParser(xml_file)

# dataframes = parser.convert_to_dataframes()

In [3]:
record_df = pd.read_csv(r'C:\Users\amank\Downloads\Fithack\record_df.csv')

C:\Users\amank\AppData\Local\Temp\ipykernel_3484\149547270.py:1: DtypeWarning: Columns (2,3,7,8,10,11,12,14,15,17,21,23,26,27,31,32,33,34,35,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  record_df = pd.read_csv(r'C:\Users\amank\Downloads\Fithack\record_df.csv')


In [4]:
record_df.tail()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device,MetadataEntry_HKMetadataKeyHeartRateMotionContext,element_type,MetadataEntry_key,MetadataEntry_value,MetadataEntry_HKMetadataKeySyncVersion,MetadataEntry_HKMetadataKeySyncIdentifier,MetadataEntry_HKMetadataKeyBarometricPressure,MetadataEntry_HKVO2MaxTestType,MetadataEntry_HKDateOfEarliestDataUsedForEstimate,MetadataEntry_HKMetadataKeyAppleDeviceCalibrated,MetadataEntry_HKMetadataKeyDevicePlacementSide,MetadataEntry_HKAlgorithmVersion,MetadataEntry_HKMetadataKeyHeartRateRecoveryActivityDuration,MetadataEntry_HKMetadataKeyHeartRateRecoveryTestType,MetadataEntry_HKMetadataKeyHeartRateRecoveryMaxObservedRecoveryHeartRate,MetadataEntry_HKMetadataKeyUserMotionContext,MetadataEntry_HKMetadataKeyHeartRateRecoveryActivityType,MetadataEntry_HKMetadataKeySessionEstimate,MetadataEntry_HKMetadataKeyMaximumLightIntensity,MetadataEntry_HKActivityType,MetadataEntry_HKPhysicalEffortEstimationType,MetadataEntry_HKIndoorWorkout,MetadataEntry_HKTimeZone,MetadataEntry_HKHeartRateEventThreshold,MetadataEntry_HKMetadataKeyAudioExposureLevel,MetadataEntry_HKLowCardioFitnessEventThreshold,MetadataEntry_HKVO2MaxValue,sub_element_type,InstantaneousBeatsPerMinute_bpm,InstantaneousBeatsPerMinute_time
1405988,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,117.0,3:10:26.99 PM
1405989,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,120.0,3:10:27.49 PM
1405990,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,124.0,3:10:27.97 PM
1405991,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,118.0,3:10:28.48 PM
1405992,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30 +0530,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,117.0,3:10:29.00 PM


In [5]:
record_df['creationDate'] = pd.to_datetime(record_df['creationDate'])
record_df['date'] = record_df['creationDate'].dt.date
record_df.tail()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device,MetadataEntry_HKMetadataKeyHeartRateMotionContext,element_type,MetadataEntry_key,MetadataEntry_value,MetadataEntry_HKMetadataKeySyncVersion,MetadataEntry_HKMetadataKeySyncIdentifier,MetadataEntry_HKMetadataKeyBarometricPressure,MetadataEntry_HKVO2MaxTestType,MetadataEntry_HKDateOfEarliestDataUsedForEstimate,MetadataEntry_HKMetadataKeyAppleDeviceCalibrated,MetadataEntry_HKMetadataKeyDevicePlacementSide,MetadataEntry_HKAlgorithmVersion,MetadataEntry_HKMetadataKeyHeartRateRecoveryActivityDuration,MetadataEntry_HKMetadataKeyHeartRateRecoveryTestType,MetadataEntry_HKMetadataKeyHeartRateRecoveryMaxObservedRecoveryHeartRate,MetadataEntry_HKMetadataKeyUserMotionContext,MetadataEntry_HKMetadataKeyHeartRateRecoveryActivityType,MetadataEntry_HKMetadataKeySessionEstimate,MetadataEntry_HKMetadataKeyMaximumLightIntensity,MetadataEntry_HKActivityType,MetadataEntry_HKPhysicalEffortEstimationType,MetadataEntry_HKIndoorWorkout,MetadataEntry_HKTimeZone,MetadataEntry_HKHeartRateEventThreshold,MetadataEntry_HKMetadataKeyAudioExposureLevel,MetadataEntry_HKLowCardioFitnessEventThreshold,MetadataEntry_HKVO2MaxValue,sub_element_type,InstantaneousBeatsPerMinute_bpm,InstantaneousBeatsPerMinute_time,date
1405988,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30+05:30,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,117.0,3:10:26.99 PM,2024-09-20
1405989,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30+05:30,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,120.0,3:10:27.49 PM,2024-09-20
1405990,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30+05:30,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,124.0,3:10:27.97 PM,2024-09-20
1405991,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30+05:30,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,118.0,3:10:28.48 PM,2024-09-20
1405992,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,rahul’s Apple Watch,11.0,ms,2024-09-20 15:10:30+05:30,2024-09-20 15:09:29 +0530,2024-09-20 15:10:29 +0530,19.8086,"<<HKDevice: 0x3025693b0>, name:Apple Watch, ma...",NaN,HeartRateVariabilityMetadataList,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InstantaneousBeatsPerMinute,117.0,3:10:29.00 PM,2024-09-20


In [10]:
def filter_data_by_date(df):
    max_date = df['date'].max()
    print(max_date)
    seven_days_ago = max_date - pd.Timedelta(days=7)
    print(seven_days_ago)
    return df[df['date'] >= seven_days_ago]

In [14]:
filtered_records_df = filter_data_by_date(record_df)
filtered_records_df.to_csv(r'C:\Users\amank\Downloads\Fithack\filtered_records_df.csv', index=False)

2024-09-20
2024-09-13


In [7]:
workout_df = pd.read_csv(r'C:\Users\amank\Downloads\Fithack\workout_df.csv')
workout_df['creationDate'] = pd.to_datetime(workout_df['creationDate'])
workout_df['date'] = workout_df['creationDate'].dt.date

In [11]:
filtered_workout_df = filter_data_by_date(workout_df)

2024-09-16
2024-09-09


In [13]:
filtered_workout_df.to_csv(r'C:\Users\amank\Downloads\Fithack\filtered_workout_df.csv', index=False)

In [17]:
activity_df = pd.read_csv(r'C:\Users\amank\Downloads\Fithack\activity_df.csv')
activity_df['dateComponents'] = pd.to_datetime(activity_df['dateComponents'])
activity_df['date'] = activity_df['dateComponents'].dt.date

In [18]:
filtered_activity_df = filter_data_by_date(activity_df)

2024-09-20
2024-09-13


In [20]:
filtered_activity_df.to_csv(r'C:\Users\amank\Downloads\Fithack\filtered_activity_df.csv', index=False)